# Creating dataframes from chess.com

In [1]:
from chessdotcom import get_titled_players, get_player_profile, get_player_stats, get_player_games_by_month
import pandas as pd
from tqdm import tqdm
import datetime
import chess.pgn
from io import StringIO
import random

## Loading information about Grandmasters

In [2]:
# Names of all chess.com accounts belonging to chess grandmasters
gm_list = get_titled_players('GM')
print(gm_list.players[:10])

['123lt', '124chess', '1977ivan', '1stsecond', '4thd-alpeacefulmoon', '64aramis64', '64arthos64', '64atilla64', '64dartagnan64', '64genghis64']


In [4]:
with open('data/gm_list.txt', 'w+') as f:
    for gm in gm_list.players:
        f.write(f"{gm}\n")

In [2]:
file = open('data/gm_list.txt', 'r')
gm_list = file.read()
gm_list = gm_list.split("\n")
file.close()

### Player profiles

In [6]:
# Pulling GMs profiles from chess.com
gm_profiles = []

for player in gm_list.players:
    gm_profiles.append(get_player_profile(player).json['player'])

In [7]:
# Change into pandas dataframe
profiles_df = pd.DataFrame(gm_profiles)

In [10]:
# Save the dataframe into csv
profiles_df.to_csv("GM_profiles.csv")

### Player stats

In [5]:
# Pulling GMs stats from chess.com
gm_stats = []

for player in tqdm(gm_list.players):
    temp_stat = get_player_stats(player, tts=2).json['stats']
    temp_stat['player'] = player
    gm_stats.append(temp_stat)

stats_df = pd.DataFrame(pd.json_normalize(gm_stats))
stats_df.to_csv("GM_stats.csv")

100%|██████████| 1466/1466 [1:09:10<00:00,  2.83s/it]


### Latest player games

In [3]:
current_time = datetime.datetime.now()

In [17]:
# Pulling GMs latest games from chess.com
gm_games = []

for player in tqdm(gm_list):
    try:
        games = get_player_games_by_month(player, year=2022, month=12, tts=random.randint(0,2))
        for game in games.json['games']:
            try:
                pgn = StringIO(game['pgn'])
                game = chess.pgn.read_game(pgn)
                game_dict = {'White': game.headers['White'], 'Black': game.headers['Black'], 'Result': game.headers['Result'], \
                'Opening': game.headers['ECO']}
                gm_games.append(game_dict)
            except:
                pass
    except:
        pass

games_df = pd.DataFrame(gm_games)
games_df.to_csv("data/GM_games.csv")

100%|██████████| 1467/1467 [41:33<00:00,  1.70s/it] 
